In [2]:
import datetime
import time

import pandas as pd
import numpy as np

!pip install torch
import torch
from sklearn.model_selection import train_test_split, KFold
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.autograd import Variable

import math

import random
import os

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.tree import DecisionTreeClassifier

In [1]:
from models.conv_lstm import ConvLSTM

# 1. EDA of the final dataset

In [3]:
df_pds = pd.read_csv('features/features_labelled_and_available_final.csv', sep = ';')

In [4]:
df_pds.head(5)

,date,pump_index,std_rush_order,avg_rush_order,std_trades,std_volume,avg_volume,std_price,avg_price,avg_price_max,hour_sin,hour_cos,minute_sin,minute_cos,symbol,Was_Pump_or_no
0,2021-03-27 02:24:00,0,0.000,0.000,-0.000,-0.0,-0.002,-0.001,0.001,0.001,0.52,0.854,0.553,-0.833,PIVX,0
1,2021-03-27 02:24:15,0,0.003,0.003,0.001,-0.0,0.001,-0.000,0.001,0.001,0.52,0.854,0.553,-0.833,PIVX,0
2,2021-03-27 02:24:30,0,0.000,0.003,-0.000,-0.0,0.001,-0.000,0.001,0.001,0.52,0.854,0.553,-0.833,PIVX,0
3,2021-03-27 02:24:45,0,0.000,0.003,-0.000,-0.0,0.000,-0.000,0.001,0.001,0.52,0.854,0.553,-0.833,PIVX,0
4,2021-03-27 02:25:00,0,0.000,0.000,-0.000,-0.0,0.000,0.000,0.001,0.001,0.52,0.854,0.461,-0.887,PIVX,0


In [5]:
df_pds[df_pds['Was_Pump_or_no'] == 1].shape

(42, 16)

In [6]:
df_pds.shape

(203652, 16)

In [7]:
df_pds.dtypes

date               object
pump_index          int64
std_rush_order    float64
avg_rush_order    float64
std_trades        float64
std_volume        float64
avg_volume        float64
std_price         float64
avg_price         float64
avg_price_max     float64
hour_sin          float64
hour_cos          float64
minute_sin        float64
minute_cos        float64
symbol             object
Was_Pump_or_no      int64
dtype: object

In [8]:
df_pds['date'] = pd.to_datetime(df_pds['date'])

In [9]:
df_pds.dtypes

date              datetime64[ns]
pump_index                 int64
std_rush_order           float64
avg_rush_order           float64
std_trades               float64
std_volume               float64
avg_volume               float64
std_price                float64
avg_price                float64
avg_price_max            float64
hour_sin                 float64
hour_cos                 float64
minute_sin               float64
minute_cos               float64
symbol                    object
Was_Pump_or_no             int64
dtype: object

In [10]:
df_pds.describe()

,pump_index,std_rush_order,avg_rush_order,std_trades,std_volume,avg_volume,std_price,avg_price,avg_price_max,hour_sin,hour_cos,minute_sin,minute_cos,Was_Pump_or_no
count,203652.000000,203652.000000,203652.000000,203652.000000,203652.000000,203652.000000,203652.000000,203652.000000,2.036520e+05,203652.000000,203652.000000,203652.000000,203652.000000,203652.000000
mean,19.144276,0.001464,0.000128,0.001844,0.002446,0.000179,-0.000037,-0.000005,-2.946202e-08,-0.129867,-0.007976,0.009242,0.023659,0.000206
std,13.130368,0.240291,0.033790,0.249516,0.330884,0.036872,0.014660,0.003606,5.094270e-03,0.697986,0.704282,0.698317,0.715339,0.014359
min,0.000000,-0.723000,-0.471000,-0.547000,-0.789000,-0.499000,-0.271000,-0.157000,-2.310000e-01,-0.998000,-0.991000,-1.000000,-0.999000,0.000000
25%,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001000,0.000000,-1.000000e-03,-0.817000,-0.776000,-0.678000,-0.698000,0.000000
50%,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-0.270000,-0.068000,0.000000,0.080000,0.000000
75%,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001000,0.000000,0.000000e+00,0.520000,0.683000,0.716000,0.735000,0.000000
max,42.000000,83.673000,10.620000,71.262000,95.778000,9.035000,4.014000,0.580000,1.222000e+00,0.998000,1.000000,1.000000,1.000000,1.000000


In [11]:
df_pds.isnull().sum()

date              0
pump_index        0
std_rush_order    0
avg_rush_order    0
std_trades        0
std_volume        0
avg_volume        0
std_price         0
avg_price         0
avg_price_max     0
hour_sin          0
hour_cos          0
minute_sin        0
minute_cos        0
symbol            0
Was_Pump_or_no    0
dtype: int64

* Well, there is no missing values, dataset is ready for the final empirical research

# 2. Empirical research, P&Ds detection

## 2.1 Classical machine learning methods

### First methodology: without train/test split

### Random Forest,  employed by La Morgia et al., 2020

In [38]:
def classifier():

    computed_data = df_pds
    time_freq = '15S'
    features = ['std_rush_order',
                'avg_rush_order',
                'std_trades',
                'std_volume',
                'avg_volume',
                'std_price',
                'avg_price',
                'avg_price_max',
                'hour_sin',
                'hour_cos',
                'minute_sin',
                'minute_cos']

    X = computed_data[features]
    Y = computed_data['Was_Pump_or_no'].astype(int).values.ravel()
    
    for number_estimators in range(50, 250, 50):
        for maximum_depth in range(4, 6, 1):
            clf = RandomForestClassifier(n_estimators=number_estimators, max_depth=maximum_depth, random_state=1)

            cv_list = [5]
            processes = 7

            for n_fold in cv_list:
                start = datetime.datetime.now()
                print('!!!For n_estimator = {} and for max_depth = {}'.format(number_estimators,maximum_depth))
                print('Processing: {} folds - time freq {}'.format(n_fold, time_freq))
                y_pred = cross_val_predict(clf, X, Y.ravel(), cv=StratifiedKFold(n_splits=n_fold), n_jobs=processes)
                print('Recall: {}'.format(recall_score(Y, y_pred)))
                print('Precision: {}'.format(precision_score(Y, y_pred)))
                print('F1 score: {}'.format(f1_score(Y, y_pred)))
                print(datetime.datetime.now() - start)

In [39]:
start = datetime.datetime.now()
classifier()
print(datetime.datetime.now() - start)

!!!For n_estimator = 50 and for max_depth = 4
Processing: 5 folds - time freq 15S
Recall: 0.7857142857142857
Precision: 0.8918918918918919
F1 score: 0.8354430379746834
0:00:02.298297
!!!For n_estimator = 50 and for max_depth = 5
Processing: 5 folds - time freq 15S
Recall: 0.7857142857142857
Precision: 0.8918918918918919
F1 score: 0.8354430379746834
0:00:02.286439
!!!For n_estimator = 100 and for max_depth = 4
Processing: 5 folds - time freq 15S
Recall: 0.7857142857142857
Precision: 0.868421052631579
F1 score: 0.825
0:00:03.755370
!!!For n_estimator = 100 and for max_depth = 5
Processing: 5 folds - time freq 15S
Recall: 0.7857142857142857
Precision: 0.8461538461538461
F1 score: 0.8148148148148148
0:00:04.086647
!!!For n_estimator = 150 and for max_depth = 4
Processing: 5 folds - time freq 15S
Recall: 0.7857142857142857
Precision: 0.8461538461538461
F1 score: 0.8148148148148148
0:00:05.504503
!!!For n_estimator = 150 and for max_depth = 5
Processing: 5 folds - time freq 15S
Recall: 0.785

### Second methodology: with split

In [143]:
def classifier_split_new():

    computed_data = df_pds
    time_freq = '15S'
    features = ['std_rush_order',
                'avg_rush_order',
                'std_trades',
                'std_volume',
                'avg_volume',
                'std_price',
                'avg_price',
                'avg_price_max',
                'hour_sin',
                'hour_cos',
                'minute_sin',
                'minute_cos',
                'pump_index']

    X = computed_data[features]
    Y = computed_data[['Was_Pump_or_no', 'pump_index']]
    #best results from the previous results --> n_estimators = 50, maximum_depth = 5
    clf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=15)

    start = datetime.datetime.now()
    print('!!!For n_estimator = {} and for max_depth = {}'.format(50,5))
    print(f'Fitting model for {time_freq}.')
    X_train, X_test = X[X['pump_index'] <= 33], X[X['pump_index'] > 33]
    Y_train, Y_test = Y[Y['pump_index'] <= 33], Y[Y['pump_index'] > 33]
    Y_train, Y_test = Y_train.drop('pump_index', axis=1), Y_test.drop('pump_index', axis=1)
    clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print('Recall: {}'.format(recall_score(Y_test, y_pred)))
    print('Precision: {}'.format(precision_score(Y_test, y_pred)))
    print('F1 score: {}'.format(f1_score(Y_test, y_pred)))
    
    rf_regressors = clf.feature_importances_
    df_regressors = pd.DataFrame({'feature_important' : rf_regressors}, index = features)
    df_regressors = df_regressors.sort_values(by = 'feature_important', ascending = False)
    return df_regressors

In [146]:
start = datetime.datetime.now()
df = classifier_split_new()
print(datetime.datetime.now() - start)

!!!For n_estimator = 50 and for max_depth = 5
Fitting model for 15S.
Recall: 0.8888888888888888
Precision: 0.7272727272727273
F1 score: 0.7999999999999999
0:00:01.375904


In [147]:
df

,feature_important
std_volume,0.294323
std_trades,0.224266
std_rush_order,0.182888
avg_volume,0.089869
avg_rush_order,0.054079
avg_price_max,0.041316
avg_price,0.041207
std_price,0.029229
hour_cos,0.012688
hour_sin,0.011201


### SVM

In [118]:
def classifier_SVM():

    computed_data = df_pds
    time_freq = '15S'
    features = ['std_rush_order',
                'avg_rush_order',
                'std_trades',
                'std_volume',
                'avg_volume',
                'std_price',
                'avg_price',
                'avg_price_max',
                'hour_sin',
                'hour_cos',
                'minute_sin',
                'minute_cos',
                'pump_index']

    X = computed_data[features]
    Y = computed_data[['Was_Pump_or_no', 'pump_index']]
    
    X_train, X_test = X[X['pump_index'] <= 33], X[X['pump_index'] > 33]
    Y_train, Y_test = Y[Y['pump_index'] <= 33], Y[Y['pump_index'] > 33]
    Y_train, Y_test = Y_train.drop('pump_index', axis=1), Y_test.drop('pump_index', axis=1)
    
    clf = svm.SVC()
    clf.fit(X_train, Y_train)
    start = datetime.datetime.now()
    y_pred = clf.predict(X_test)
    print('Recall: {}'.format(recall_score(Y_test, y_pred)))
    print('Precision: {}'.format(precision_score(Y_test, y_pred)))
    print('F1 score: {}'.format(f1_score(Y_test, y_pred)))
    print(datetime.datetime.now() - start)

In [119]:
start = datetime.datetime.now()
classifier_SVM()
print(datetime.datetime.now() - start)

Recall: 0.3333333333333333
Precision: 1.0
F1 score: 0.5
0:00:00.250024
0:00:00.945531


# Data preparing for deep learning approach 

* Key parameters 

In [18]:
batch_size = 600
segment_length = 15
undersample_ratio = 0.05
train_ratio = 0.8

In [19]:
def get_pumps(data, segment_length, *, pad=True):
    pumps = []
    skipped_row_count = 0
    for pump_index in np.unique(data['pump_index'].values):
        pump_i = data[data['pump_index'] == pump_index].copy()
        if len(pump_i) < MIN_PUMP_SIZE:
            print(f'Pump {pump_index} has {len(pump_i)} rows, skipping')
            skipped_row_count += len(pump_i)
            continue
        pump_i['delta_minutes'] = (pump_i['date'] - pump_i['date'].shift(1)).fillna(PLACEHOLDER_TIMEDELTA)
        pump_i['delta_minutes'] = pump_i['delta_minutes'].apply(lambda x: x.total_seconds() / 60)
        pump_i = pump_i[FEATURE_NAMES + ['Was_Pump_or_no']]
        pump_i = pump_i.values.astype(np.float32)
        if pad:
            pump_i = np.pad(pump_i, ((segment_length - 1, 0), (0, 0)), 'reflect')
        pumps.append(pump_i)
    print(f'Skipped {skipped_row_count} rows total')
    print(f'{len(pumps)} pumps')
    return pumps


In [20]:
PLACEHOLDER_TIMEDELTA = pd.Timedelta(minutes=0)
MIN_PUMP_SIZE = 100
FEATURE_NAMES = [
    'std_rush_order',
    'avg_rush_order',
    'std_trades',
    'std_volume',
    'avg_volume',
    'std_price',
    'avg_price',
    'avg_price_max',
    'hour_sin',
    'hour_cos',
    'minute_sin',
    'minute_cos',
    'delta_minutes',
]

get_pumps(df_pds, segment_length)

Skipped 0 rows total
43 pumps


[array([[-0.   , -0.003,  0.   , ..., -0.999,  0.25 ,  0.   ],
        [-0.   , -0.003,  0.   , ..., -0.987,  0.5  ,  0.   ],
        [ 0.   ,  0.   ,  0.   , ..., -0.987,  0.25 ,  0.   ],
        ...,
        [ 0.   ,  0.   , -0.   , ...,  1.   ,  0.25 ,  0.   ],
        [ 0.001,  0.004,  0.   , ...,  1.   ,  0.75 ,  0.   ],
        [ 0.001,  0.004,  0.001, ...,  1.   ,  0.25 ,  0.   ]],
       dtype=float32),
 array([[ 0.002,  0.004, -0.   , ..., -0.887,  0.25 ,  0.   ],
        [ 0.   ,  0.   ,  0.   , ..., -0.931,  0.25 ,  0.   ],
        [ 0.   ,  0.   , -0.   , ..., -0.931,  0.25 ,  0.   ],
        ...,
        [ 0.   ,  0.   ,  0.   , ...,  0.994,  0.5  ,  0.   ],
        [ 0.001,  0.004, -0.   , ...,  1.   ,  0.25 ,  0.   ],
        [ 0.   ,  0.   ,  0.   , ...,  1.   ,  1.   ,  0.   ]],
       dtype=float32),
 array([[ 0.00e+00,  0.00e+00,  0.00e+00, ...,  4.85e-01,  2.00e+00,
          0.00e+00],
        [ 0.00e+00,  0.00e+00,  0.00e+00, ...,  6.59e-01,  1.75e+00,
          0

In [21]:
def process_data(data, *, segment_length, remove_post_anomaly_data=False):
    print('Processing data...')
    print(f'Segment length: {segment_length}')
    print(f'Remove post anomaly data: {remove_post_anomaly_data}')
    print(f'Data shape: {data.shape}')
    pumps = get_pumps(data, segment_length)
    segments = []
    remove_cnt = 0
    for pump in pumps:
        for i, window in enumerate(np.lib.stride_tricks.sliding_window_view(pump, segment_length, axis=0)):
            segment = window.transpose()
            if remove_post_anomaly_data and segment[:-1, -1].sum() > 0:
                remove_cnt += 1
                continue
            segments.append(segment)
    if remove_post_anomaly_data:
        print(f'Removed {remove_cnt} rows with post-anomaly data')
    print(f'{len(segments)} rows of data after processing')
    return np.stack(segments)

In [22]:
data = process_data(df_pds, segment_length=segment_length)
data

Processing data...
Segment length: 15
Remove post anomaly data: False
Data shape: (203652, 16)
Skipped 0 rows total
43 pumps
203652 rows of data after processing


array([[[-0.00e+00, -3.00e-03,  0.00e+00, ..., -9.99e-01,  2.50e-01,
          0.00e+00],
        [-0.00e+00, -3.00e-03,  0.00e+00, ..., -9.87e-01,  5.00e-01,
          0.00e+00],
        [ 0.00e+00,  0.00e+00,  0.00e+00, ..., -9.87e-01,  2.50e-01,
          0.00e+00],
        ...,
        [ 0.00e+00,  3.00e-03, -0.00e+00, ..., -8.33e-01,  2.50e-01,
          0.00e+00],
        [ 3.00e-03,  3.00e-03,  1.00e-03, ..., -8.33e-01,  2.50e-01,
          0.00e+00],
        [ 0.00e+00,  0.00e+00, -0.00e+00, ..., -8.33e-01,  0.00e+00,
          0.00e+00]],

       [[-0.00e+00, -3.00e-03,  0.00e+00, ..., -9.87e-01,  5.00e-01,
          0.00e+00],
        [ 0.00e+00,  0.00e+00,  0.00e+00, ..., -9.87e-01,  2.50e-01,
          0.00e+00],
        [ 0.00e+00,  0.00e+00, -0.00e+00, ..., -9.87e-01,  2.50e-01,
          0.00e+00],
        ...,
        [ 3.00e-03,  3.00e-03,  1.00e-03, ..., -8.33e-01,  2.50e-01,
          0.00e+00],
        [ 0.00e+00,  0.00e+00, -0.00e+00, ..., -8.33e-01,  0.00e+00,
   

In [23]:
def undersample_train_data(train_data, undersample_ratio):
    with_anomalies = train_data[:, :, -1].sum(axis=1) > 0
    mask = with_anomalies | (np.random.rand(train_data.shape[0]) < undersample_ratio)
    return train_data[mask]

def create_loaders(data, *, train_ratio, batch_size, undersample_ratio):
    '''
    creates train and test loaders given a list of np-array pumps of equal length
    '''
    # split into train/validate; return dataloaders for each set
    train_data, test_data = train_test_split(data, train_size=train_ratio, shuffle=False)
    print(f'Train data shape: {train_data.shape}')
    train_data = undersample_train_data(train_data, undersample_ratio)
    print(f'Train data shape after undersampling: {train_data.shape}')
    print(f'Test data shape: {test_data.shape}')
    print(f'{int(train_data[:, -1, -1].sum())} segments in train data ending in anomaly')
    print(f'{int(test_data[:, -1, -1].sum())} segments in test data ending in anomaly')
    train_data, test_data = torch.FloatTensor(train_data), torch.FloatTensor(test_data)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_data, batch_size=500)#, drop_last=True)
    return train_loader, test_loader


In [24]:
train_loader, test_loader = create_loaders(data, train_ratio = train_ratio, batch_size=batch_size, 
                                           undersample_ratio=undersample_ratio)

Train data shape: (162921, 15, 14)
Train data shape after undersampling: (8631, 15, 14)
Test data shape: (40731, 15, 14)
33 segments in train data ending in anomaly
9 segments in test data ending in anomaly


## C-LSTM ,  employed by Chadalapaka et al., 2022

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

* Key Parameters 

In [26]:
n_epochs = 50
time_epochs = True
validate_every_n = 10
final_run = False
prthreshold_1 = 0.4
verbose = False
lr_decay_step = 0
n_feats = 13
kernel_size = 3
embedding_size = 350
n_layers = 1
dropout = 0.0
cell_norm = False
out_norm = False
train_output_every_n = 5
lr = 1e-3
weight_decay = 0.0
undersample_ratio = 0.05

In [27]:
def collect_metrics_n_epochs(model, *, train_loader, test_loader,
                            optimizer, criterion, device, lr_scheduler=None, feature_count=13):
    prthreshold_1 = 0.4
    best_metrics = np.array([0.0]*4)
    for epoch in range(n_epochs):
        start = time.time()
        loss = train(model, train_loader, optimizer, criterion, device, feature_count)
        if (epoch + 1) % train_output_every_n == 0:
            print(f'Epoch {epoch + 1}{f" ({(time.time()-start):0.2f}s)" if time_epochs else ""} -- Train Loss: {loss:0.5f}')
        if (epoch + 1) % validate_every_n == 0 or final_run:
            if prthreshold_1 > 0:
                prthreshold = prthreshold_1
            else:
                prthreshold = pick_threshold(model, train_loader, undersample_ratio, device, verbose=verbose, feature_count=feature_count)
            print(prthreshold)
            acc, precision, recall, f1 = validate(model, test_loader, device, verbose=verbose, pr_threshold=prthreshold, feature_count=feature_count)
            if f1 > best_metrics[-1]:
                best_metrics = [acc, precision, recall, f1]
            print(f'Val   -- Acc: {acc:0.5f} -- Precision: {precision:0.5f} -- Recall: {recall:0.5f} -- F1: {f1:0.5f}')
        if lr_decay_step > 0 and (epoch+1) % lr_decay_step == 0:
            if lr_scheduler: lr_scheduler.step(epoch+1)
    return best_metrics

def train(model, dataloader, opt, criterion, device, feature_count=13):
    '''
    trains given model with given dataloader, optimizer, criterion, and on device.
    :returns: avg loss/batch for this epoch
    '''
    epoch_loss = 0
    for batch in dataloader:
        opt.zero_grad()
        x = batch[:, :, :feature_count].to(device)
        y = batch[:, :, -1].to(device)
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        opt.step()
        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

def validate(model, dataloader, device, verbose=True, pr_threshold=0.7, criterion=None, feature_count=13):
    preds_1 = []
    preds_0 = []
    all_ys = []
    all_preds = []
    epoch_loss = 0
    for batch in dataloader:
        with torch.no_grad():
            # only consider the last chunk of each segment for validation
            x = batch[:, :, :feature_count].to(device)
            y = batch[:, -1, -1].to(device)
            preds = model(x)[:, -1]
            y, preds = y.cpu().flatten(), preds.cpu().flatten()
            if verbose:
                preds_0.extend(preds[y == 0])
                preds_1.extend(preds[y == 1])
            all_ys.append(y)
            all_preds.append(preds)
            if criterion is not None:
                loss = criterion(preds, y)
                epoch_loss += loss.item()
    if verbose:
        print(f'Mean output at 0: {(sum(preds_0) / len(preds_0)).item():0.5f} at 1: {(sum(preds_1) / len(preds_1)).item():0.5f}')
    y = torch.cat(all_ys, dim=0).cpu()
    preds = torch.cat(all_preds, dim=0).cpu()
    preds = preds >= pr_threshold
    acc = accuracy_score(y, preds)
    precision = precision_score(y, preds, zero_division=0)
    recall = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)
    if criterion is not None:
        return acc, precision, recall, f1, epoch_loss/len(dataloader)
    else:
        return acc, precision, recall, f1

def pick_threshold(model, dataloader, undersample_ratio, device, verbose=True, feature_count=13):
    all_ys = []
    all_preds = []
    for batch in dataloader:
        with torch.no_grad():
            # only consider the last chunk of each segment for validation
            x = batch[:, :, :feature_count].to(device)
            y = batch[:, -1, -1].to(device)
            preds = model(x)[:, -1]
            y, preds = y.cpu().flatten(), preds.cpu().flatten()
            all_ys.append(y)
            all_preds.append(preds)
    y = torch.cat(all_ys, dim=0).cpu()
    preds = torch.cat(all_preds, dim=0).cpu()
    y = y.numpy()
    preds = preds.numpy()
    _, _, thresholds = precision_recall_curve(y, preds)

    best_f1 = 0
    best_threshold = 0
    for threshold in thresholds:
        true_pos = np.sum(preds[y == 1] >= threshold)
        false_pos = np.sum(preds[y == 0] >= threshold)
        false_neg = np.sum(preds[y == 1] < threshold)
        true_neg = np.sum(preds[y == 0] < threshold)

        false_pos /= undersample_ratio
        true_neg /= undersample_ratio

        precision = true_pos / (true_pos + false_pos)
        recall = true_pos / (true_pos + false_neg)
        f1 = 2 * precision * recall / (precision + recall)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    if verbose:
        print(f'Best threshold: {best_threshold} (train f1: {best_f1})')

    return best_threshold

In [28]:
criterion = torch.nn.BCELoss().to(device)

In [29]:
def create_conv_model():
    return ConvLSTM(n_feats, kernel_size, embedding_size, n_layers, dropout=dropout,
        cell_norm=cell_norm, out_norm=out_norm).to(device)

In [30]:
model = create_conv_model()

In [31]:
model

ConvLSTM(
  (conv1): Conv1d(13, 350, kernel_size=(3,), stride=(1,))
  (relu1): ReLU()
  (pool): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (lstm): LSTM(350, 350, batch_first=True)
  (ln): LayerNorm((350,), eps=1e-05, elementwise_affine=True)
  (o_proj): Linear(in_features=350, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [32]:
# reproducability
seed = 0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
g = torch.Generator()
g.manual_seed(seed)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:2'

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

best_metrics = collect_metrics_n_epochs(
    model,
    train_loader=train_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    feature_count=n_feats
)
fold_metrics = np.array([0.0]*4)
fold_metrics += np.array(best_metrics)
print(f'Best F1 this run: {best_metrics[-1]}')
print()

Epoch 5 (7.80s) -- Train Loss: 0.02152
Epoch 10 (7.85s) -- Train Loss: 0.00735
0.4
Val   -- Acc: 0.99983 -- Precision: 0.75000 -- Recall: 0.33333 -- F1: 0.46154
Epoch 15 (9.64s) -- Train Loss: 0.00299
Epoch 20 (8.55s) -- Train Loss: 0.00203
0.4
Val   -- Acc: 0.99993 -- Precision: 0.80000 -- Recall: 0.88889 -- F1: 0.84211
Epoch 25 (9.69s) -- Train Loss: 0.00169
Epoch 30 (8.59s) -- Train Loss: 0.00152
0.4
Val   -- Acc: 0.99993 -- Precision: 0.80000 -- Recall: 0.88889 -- F1: 0.84211
Epoch 35 (8.96s) -- Train Loss: 0.00142
Epoch 40 (8.56s) -- Train Loss: 0.00129
0.4
Val   -- Acc: 0.99990 -- Precision: 0.72727 -- Recall: 0.88889 -- F1: 0.80000
Epoch 45 (8.25s) -- Train Loss: 0.00131
Epoch 50 (8.48s) -- Train Loss: 0.00122
0.4
Val   -- Acc: 0.99990 -- Precision: 0.72727 -- Recall: 0.88889 -- F1: 0.80000
Best F1 this run: 0.8421052631578948



## TranAD model creation 

In [266]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model).float() * (-math.log(10000.0) / d_model))
        pe += torch.sin(position * div_term)
        pe += torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x, pos=0):
        x = x + self.pe[pos:pos+x.size(0), :]
        return self.dropout(x)
    
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=16, dropout=0):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = nn.LeakyReLU(True)

    def forward(self, src,src_mask=None, src_key_padding_mask=None):
        src2 = self.self_attn(src, src, src)[0]
        src = src + self.dropout1(src2)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        return src
    
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=16, dropout=0):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = nn.LeakyReLU(True)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        tgt2 = self.self_attn(tgt, tgt, tgt)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt2 = self.multihead_attn(tgt, memory, memory)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        return tgt

In [316]:
# Proposed Model (VLDB 22)
class TranAD_Basic(nn.Module):
	def __init__(self):
		super(TranAD_Basic, self).__init__()
		self.name = 'TranAD_Basic'
		self.lr = lr
		self.batch = 600
		self.n_feats = 14
		self.n_window = 10
		self.n = self.n_feats * self.n_window
		self.pos_encoder = PositionalEncoding(self.n_feats, 0.1, self.n_window)
		encoder_layers = TransformerEncoderLayer(d_model=self.n_feats, nhead=self.n_feats, dim_feedforward=16, dropout=0.1)
		self.transformer_encoder = TransformerEncoder(encoder_layers, 1)
		decoder_layers = TransformerDecoderLayer(d_model=self.n_feats, nhead=self.n_feats, dim_feedforward=16, dropout=0.1)
		self.transformer_decoder = TransformerDecoder(decoder_layers, 1)
		self.fcn = nn.Sigmoid()

	def forward(self, src, tgt):
		src = src * math.sqrt(self.n_feats)
		src = self.pos_encoder(src)
		memory = self.transformer_encoder(src)
		x = self.transformer_decoder(tgt, memory)
		x = self.fcn(x)
		return x

In [317]:
model_1 = TranAD_Basic()

In [318]:
model_1

TranAD_Basic(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=14, out_features=14, bias=True)
        )
        (linear1): Linear(in_features=14, out_features=16, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=16, out_features=14, bias=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (activation): LeakyReLU(negative_slope=True)
      )
    )
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0): TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=14, out_features=14, bias=True)
        )
        (multihead_attn): Mu

In [319]:
test_loader, train_loader

(<torch.utils.data.dataloader.DataLoader at 0x7f91d306b070>,
 <torch.utils.data.dataloader.DataLoader at 0x7f9214ce20d0>)

In [301]:
num_epochs = 10
feature_count = 13

In [302]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss().to(device)

for epoch in range(num_epochs):
    for batch in train_loader:
        src = batch[:, :, :feature_count].to(device)
        tgt = batch[:, -1, -1].to(device)
        
        optimizer.zero_grad()
        
        output = model_1(src, tgt)
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

RuntimeError: The size of tensor a (600) must match the size of tensor b (10) at non-singleton dimension 0

In [303]:
def train(model, train_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()
            sct = batch[:, :, :feature_count-1].to(device)
            tgt = batch[:, -1, -1].to(device)
            outputs = model(sct,tgt)
            loss = criterion(outputs, tgt)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('Epoch [%d]/[%d] loss: %.3f' % (epoch+1, num_epochs, running_loss))

def test(model, test_loader):
    y_true = []
    y_pred = []
    with no_grad():
        for batch in test_loader:
            sct = batch[:, :, :feature_count-1].to(device)
            tgt = batch[:, -1, -1].to(device)
            outputs = model(sct)
            _, predicted = torch.max(outputs.data, 1)
            y_true += list(tgt.cpu().numpy())
            y_pred += list(predicted.cpu().numpy())
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    print('F1 Score: %.3f' % f1)
    print('Precision: %.3f' % precision)
    print('Recall: %.3f' % recall)


In [304]:
# Конфигурация модели и параметров обучения
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
feature_count = 14
model = model_1
num_epochs = 10

# Обучение модели
train(model, train_loader, optimizer, criterion, num_epochs)

# Тестирование модели
test(model, test_loader)

RuntimeError: The size of tensor a (600) must match the size of tensor b (10) at non-singleton dimension 0

In [ ]:
class TranAD_Basic(nn.Module):
    def __init__(self, feats):
        super(TranAD_Basic, self).__init__()
        self.name = 'TranAD_Basic'
        self.lr = lr
        self.batch = 128
        self.n_feats = feats
        self.n_window = 10
        self.n = self.n_feats * self.n_window
        self.pos_encoder = PositionalEncoding(feats, 0.1, self.n_window)
        encoder_layers = TransformerEncoderLayer(d_model=feats, nhead=feats, dim_feedforward=16, dropout=0.1)
        self.transformer_encoder = TransformerEncoder(encoder_layers, 1)
        decoder_layers = TransformerDecoderLayer(d_model=feats, nhead=feats, dim_feedforward=16, dropout=0.1)
        self.transformer_decoder = TransformerDecoder(decoder_layers, 1)
        self.fcn = nn.Sigmoid()

    def forward(self, src, tgt):
        src = src * math.sqrt(self.n_feats)
        src = self.pos_encoder(src)
        memory = self.transformer_encoder(src)
        x = self.transformer_decoder(tgt, memory)
        x = self.fcn(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TranAD_Basic(feats=13).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss().to(device)

def train(model, train_loader, optimizer, criterion):
    model.train()
    for batch_idx in train_loader:
        X = batch[:, :, :feature_count].to(device)
        y = batch[:, -1, -1].to(device)
        optimizer.zero_grad()
        output = model(X, y)
        loss = criterion(output.squeeze(), y)
        loss.backward()
        optimizer.step()

def test(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            output = model(X[:, :-1, :].transpose(1, 2), X[:, -1, :])
            output = (output > 0.5).float()
            y_true += list(y.cpu().numpy())
            y_pred += list(output.squeeze().cpu().numpy())
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    print('F1 Score: %.3f' % f1)
    print('Precision: %.3f' % precision)
    print('Recall: %.3f' % recall)

num_epochs = 10
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion)
    test(model, test_loader)

In [326]:
class TranAD_Basic(nn.Module):
    def __init__(self, feats):
        super(TranAD_Basic, self).__init__()
        self.name = 'TranAD_Basic'
        self.lr = lr
        self.batch = 128
        self.n_feats = feats
        self.n_window = 10
        self.n = self.n_feats * self.n_window
        self.pos_encoder = PositionalEncoding(feats, 0.1, self.n_window)
        encoder_layers = TransformerEncoderLayer(d_model=feats, nhead=feats, dim_feedforward=16, dropout=0.1)
        self.transformer_encoder = TransformerEncoder(encoder_layers, 1)
        decoder_layers = TransformerDecoderLayer(d_model=feats, nhead=feats, dim_feedforward=16, dropout=0.1)
        self.transformer_decoder = TransformerDecoder(decoder_layers, 1)
        self.fcn = nn.Sigmoid()

    def forward(self, src, tgt):
        src = src * math.sqrt(self.n_feats)
        src = self.pos_encoder(src)
        memory = self.transformer_encoder(src)
        x = self.transformer_decoder(tgt, memory)
        x = self.fcn(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TranAD_Basic(feats=14).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss().to(device)

def train(model, train_loader, optimizer, criterion):
    model.train()
    for batch_idx in train_loader:
        X = batch[:, :, :feature_count].to(device)
        y = batch[:, -1:, -1].to(device)
        optimizer.zero_grad()
        output = model(X, y)
        loss = criterion(output.squeeze(), y)
        loss.backward()
        optimizer.step()

In [327]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model).float() * (-math.log(10000.0) / d_model))
        pe += torch.sin(position * div_term)
        pe += torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.float32, device=x.device).reshape(1, seq_len, 1)
        encodings = self.encoding(pos)
        x = x + encodings
        return self.dropout(x)
num_epochs = 10
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion)


RuntimeError: The size of tensor a (600) must match the size of tensor b (10) at non-singleton dimension 0